In [1]:
from IPython.display import GeoJSON
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings

## The code threw up some SettingWithCopyWarnings that I will fix at somepoint
warnings.filterwarnings('ignore')

In [2]:
sensorList = ['PER_AIRMON_MONITOR1048100',
                'PER_AIRMON_MONITOR1156100', 
                'PER_AIRMON_MONITOR1056100',
                'PER_AIRMON_MONITOR914', 
                'PER_AIRMON_MONITOR1155100', 
                'PER_AIRMON_MONITOR1135100']

## Key Variables - easy to add more plots
variables = ['NO2','PM2.5','PM10']

NE_Agg_DF = pd.DataFrame()

In [7]:
for s in sensorList:
    dataLoad = pd.read_pickle('C:/Covid_Baseline/recent_aq_'+s+'.pkl')
    NE_Agg_DF = NE_Agg_DF.append(dataLoad, ignore_index=True)
    
NE_Agg_DF = NE_Agg_DF[NE_Agg_DF["Flagged as Suspect Reading"]==False]
NE_Agg_DF.loc[:,"Weekday"] = pd.to_datetime(NE_Agg_DF['Timestamp']).dt.weekday
NE_Agg_DF.loc[:,"Hour"] = pd.to_datetime(NE_Agg_DF['Timestamp']).dt.hour
NE_Agg_DF.drop(columns=['Flagged as Suspect Reading','Location (WKT)','Ground Height Above Sea Level',
                            'Sensor Height Above Ground','Broker Name','Third Party','Sensor Centroid Longitude',
                            'Sensor Centroid Latitude','Raw ID'], axis=1, inplace=True)

,Sensor Name,Variable,Units,Timestamp,Value,Weekday,Hour
0,PER_AIRMON_MONITOR1048100,CO,ugm -3,2020-03-16 00:15:00,0.172895,0,0
1,PER_AIRMON_MONITOR1048100,PM1,ugm -3,2020-03-16 00:15:00,1.675000,0,0
2,PER_AIRMON_MONITOR1048100,Humidity,%,2020-03-16 00:15:00,83.597000,0,0
3,PER_AIRMON_MONITOR1048100,NO,ugm -3,2020-03-16 00:15:00,0.978750,0,0
4,PER_AIRMON_MONITOR1048100,NO2,ugm -3,2020-03-16 00:15:00,3.669760,0,0
...,...,...,...,...,...,...,...
72839,PER_AIRMON_MONITOR1135100,Pressure,hpa,2020-03-20 08:45:00,1026.000000,4,8
72840,PER_AIRMON_MONITOR1135100,Temperature,Celsius,2020-03-20 08:45:00,3.551000,4,8
72841,PER_AIRMON_MONITOR1135100,PM1,ugm -3,2020-03-20 08:45:00,5.454000,4,8
72842,PER_AIRMON_MONITOR1135100,NOx,ppb,2020-03-20 08:45:00,47.521000,4,8


In [9]:
nrows = len(variables)
fig, axs = plt.subplots(nrows,1, figsize=(30,30))
row=0

for v in variables:
    baseline = pd.read_pickle('C:/Covid_Baseline/NE_Aggregated_'+v+'_Baseline.pkl')
    
    dfCovidTemp = NE_Agg_DF[NE_Agg_DF["Variable"]==v]
    dfCovidTemp.drop(columns=['Sensor Name','Variable','Units','Timestamp'], axis=1, inplace=True)

    aggregateColumns = ['Weekday', 'Hour']
    dfCovidMean = dfCovidTemp.groupby(aggregateColumns, group_keys=False, as_index=False).median()
    dfCovidLQ = dfCovidTemp.groupby(aggregateColumns, group_keys=False, as_index=False).quantile(.15)
    dfCovidHQ = dfCovidTemp.groupby(aggregateColumns, group_keys=False, as_index=False).quantile(.85)
    
    plt.axes(axs[row])
    plt.title('North-East Aggregated : ' + v, fontsize=23)
    plt.xlim(0,167)
    plt.xlabel('Day/Hour', fontsize=18)
    plt.xticks(ticks=[0,12,24,36,48,60,72,84,96,108,120,132,144,156,168], 
                   labels=['Mon 00','Mon 12','Tues 00','Tues 12','Wed 00','Wed 12','Thurs 00','Thurs 12',
                           'Fri 00','Fri 12','Sat 00','Sat 12','Sun 00','Sun 12'], fontsize=16)

    plt.ylabel(v, fontsize=18)
    plt.plot(baseline["Mean"], color = "darkred",label="Baseline Median (Nov-Feb)")
    plt.plot(baseline["LQ"], color = "salmon",linestyle='dashed',label="Baseline P15-P85 (Nov-Feb)")
    plt.plot(baseline["HQ"], color = "salmon",linestyle='dashed')
    plt.plot(dfCovidMean["Value"], color = "navy",label="Post 16th March Median")
    plt.plot(dfCovidLQ["Value"], color = "skyblue",linestyle='dashed',label="Post 16th March P15-P85 (Nov-Feb)")
    plt.plot(dfCovidHQ["Value"], color = "skyblue",linestyle='dashed')
        
    plt.legend(loc="upper right",fontsize=18)
    
    row = row+1
    
fig.tight_layout()
## Update filepath
plt.savefig('C:/Covid_Baseline/Images/Ne_Agg_TS.png')
plt.clf()

<Figure size 2160x2160 with 0 Axes>